In [19]:
import json
import numpy as np
from collections import deque
import logging

# Configure logging
logging.basicConfig(filename='transportation_problem.log',
                    level=logging.INFO,
                    format='%(message)s')
def clean_log_file(file_path):
    with open(file_path, 'w') as file:
        file.write('')

# Example usage
clean_log_file('transportation_problem.log')

In [20]:
def visualize_allocation(allocation, supply, demand, i, j):
    logging.info("\nCurrent Allocation Matrix:")
    logging.info(" " * 10 + "   ".join([f"Dest {j + 1}" for j in range(allocation.shape[1])]))  # Header for destinations
    logging.info(" " * 10 + ("------" + "   ") * (allocation.shape[1]))  # Divider line
    for idx, all in enumerate(allocation):
        allocation_row = "| ".join(f"{int(val):>4}" + "   " for val in all)
        logging.info(f"Source {idx + 1} | " + allocation_row)
    
    logging.info(" " * 8 + f"Current allocation: {int(allocation[i, j])} at Source {i + 1}, Dest {j + 1}")

    logging.info("\nRemaining Supply:")
    logging.info(" " * 8 + " | ".join(f"{int(val):>3}" for val in supply))

    logging.info("\nRemaining Demand:")
    logging.info(" " * 8 + " | ".join(f"{int(val):>3}" for val in demand))


In [21]:
def north_west_corner_method(supply, demand):  
    supply = np.array(supply)
    demand = np.array(demand)
    
    # Number of sources and destinations
    n_sources = len(supply)
    n_destinations = len(demand)
    
    # Initialize the allocation matrix with zeros
    allocation = np.zeros((n_sources, n_destinations))
    
    # Indices for sources and destinations
    i, j = 0, 0
    # logging.info("\nCurrent Allocation Matrix:")
    # logging.info(" " * 10 + "   ".join([f"Dest {j + 1}" for j in range(allocation.shape[1])]))  # Header for destinations
    # logging.info(" " * 10 + ("------" + "   ") * (allocation.shape[1]))  # Divider line
    # for idx, all in enumerate(allocation):
    #     allocation_row = "| ".join(f"{int(val):>4}" + "   " for val in all)
    #     logging.info(f"Source {idx + 1} | " + allocation_row)

    # logging.info("\nRemaining Supply:")
    # logging.info(" " * 8 + " | ".join(f"{int(val):>3}" for val in supply))

    # logging.info("\nRemaining Demand:")
    # logging.info(" " * 8 + " | ".join(f"{int(val):>3}" for val in demand))

    # Loop until all supply and demand are satisfied
    while np.sum(supply) != 0 or np.sum(demand) != 0:
    
        # Find the minimum of the current supply and demand
        min_value = min(supply[i], demand[j])
        
        # Allocate that amount to the current cell
        allocation[i, j] = min_value
        
        # Reduce the supply and demand by the allocated amount
        supply[i] -= min_value
        demand[j] -= min_value
         
        # Visualize the current allocation matrix
        # visualize_allocation(allocation, supply, demand, i, j)

        # Move to the next row/column
        if supply[i] == 0 and i < n_sources - 1:
            i += 1 
        elif demand[j] == 0 and j < n_destinations - 1:
            j += 1
        if np.sum(supply) == 0:
            if np.sum(demand) != 0:
                logging.info("!!!!! The warehouse is empty, but there are still demands !!!!!")
                break
    
    # Final output of the allocation matrix
    logging.info("\nFinal Allocation Matrix:")
    logging.info(" " * 8 + " ".join([f"Dest {j}" for j in range(n_destinations)]))  # Header for destinations
    logging.info(" " * 6 + "-" * (n_destinations * 5))  # Divider line
    for idx, all in enumerate(allocation):
        logging.info(f"Source {idx + 1} | " + " | ".join(f"{int(val):>3}" for val in all))
    
    return allocation

In [22]:
def read_data_from_json(file_path):
    logging.info(f"Reading data from {file_path}")
    with open(file_path, 'r') as f:
        data = json.load(f)

    supply = data['supply']
    demand = data['demand']
    cost = data['cost']
    
    logging.info(f"Supply: {supply}")
    logging.info(f"Demand: {demand}")
    logging.info("Cost matrix:")
    for cs in cost:
        logging.info(f"\t{cs}")
    
    return supply, demand, cost

In [23]:
def calculate_total_cost(allocation, cost):
    allocation = np.array(allocation)
    cost = np.array(cost)
    
    # Replace NaN values with 0 in the allocation matrix
    allocation = np.nan_to_num(allocation, nan=0.0)
    
    # Element-wise multiplication of allocation and cost matrix, then sum all elements
    total_cost = np.sum(allocation * cost)
        
    return total_cost

In [24]:
def read_data_from_json(file_path):
    logging.info(f"Reading data from {file_path}")
    with open(file_path, 'r') as f:
        data = json.load(f)

    return data['data']

In [25]:
file_path = "data.json"

data = read_data_from_json(file_path)

# for index, example in enumerate(data):
#     logging.info(f"\nProcessing Example {index + 1}:")
#     supply = example['supply']
#     demand = example['demand']
#     cost = example['cost']
#     logging.info(f"Supply:\n\t{supply}")
#     logging.info(f"Demand:\n\t{demand}")
#     logging.info("Cost:")
#     for a in cost:
#         logging.info(f"\t{a}")
#     # Get the allocation table using North-West Corner Method
#     allocation = north_west_corner_method(supply, demand)

#     # Calculate the total cost
#     total_cost = calculate_total_cost(allocation, cost)
#     logging.info(f"\n ~~~~~~~~~~~~~~~ Total Transportation Cost: {total_cost} ~~~~~~~~~~~~~~~")

In [26]:
def calculate_potentials(allocation, cost):
    """
    Calculate the potentials (u and v) using the allocated cells.
    
    Parameters:
    - allocation: 2D array, the allocation matrix.
    - cost: 2D array, the cost matrix.
    
    Returns:
    - u: list, potentials for rows.
    - v: list, potentials for columns.
    """
    n_rows, n_cols = allocation.shape
    u = [None] * n_rows  # Potentials for rows (sources)
    v = [None] * n_cols  # Potentials for columns (destinations)
    
    # Initialize potentials
    u[0] = 0  # Set the first row potential to 0
    
    while None in u or None in v:
        for i in range(n_rows):
            for j in range(n_cols):
                if allocation[i, j] >= 0:  # Only consider allocated cells
                    
                    if u[i] is not None and v[j] is None:
                        v[j] = cost[i][j] - u[i]
                    
                    elif u[i] is None and v[j] is not None:
                        u[i] = cost[i][j] - v[j]
                    
    logging.info("\nCalculating potentials:")
    logging.info(f"u (rows): {u}")
    logging.info(f"v (columns): {v}")
    
    return u, v

In [27]:
def calculate_opportunity_costs(allocation, cost, u, v):
    """
    Calculate the opportunity cost (Delta_ij) for unallocated cells.
    
    Parameters:
    - allocation: 2D array, the allocation matrix.
    - cost: 2D array, the cost matrix (can be a list of lists or NumPy array).
    - u: list, potentials for rows.
    - v: list, potentials for columns.
    
    Returns:
    - delta: 2D array of opportunity costs.
    """
    n_rows, n_cols = allocation.shape
    delta = np.zeros_like(allocation)
    
    for i in range(n_rows):
        for j in range(n_cols):
            if not (allocation[i, j] >= 0):  # Only calculate for unallocated cells
                delta[i, j] = (u[i] + v[j]) - cost[i][j]
                logging.info(f"Delta[{i}, {j}]: {delta[i,j]}")
    logging.info("")
    return delta

In [28]:
def find_most_negative_delta(delta):
    """
    Find the unallocated cell with the most negative opportunity cost.
    
    Parameters:
    - delta: 2D array, the opportunity cost matrix.
    
    Returns:
    - (i, j): The coordinates of the cell with the most negative delta.
    """    
    # Log the delta matrix
    logging.info(f"Opportunity cost matrix (Delta):\n{delta}")
    
    # Find the minimum value in the delta matrix
    max_delta = np.max(delta)
    logging.info(f"Maximum opportunity cost (Delta): {max_delta}")
    
    # Check if the maximum value is negative
    if max_delta <= 0:
        logging.info("No positive opportunity costs found, solution is optimal.")
        return None  # No negative delta, solution is optimal
    else:
        # Find the coordinates of the cell with the most negative delta
        i, j = np.unravel_index(np.argmax(delta),    delta.shape)
        return i, j


In [29]:
def is_within_bounds(i, j, allocation):
    n_sources, n_destinations = allocation.shape
    return 0 <= i < n_sources and 0 <= j < n_destinations

In [30]:
def find_next_moves(i, j, allocation, move_type):
    next_moves = []
    n_sources, n_destinations = allocation.shape
    
    if move_type == 'horizontal':
        for dj in range(1, n_destinations):
            if is_within_bounds(i, j + dj, allocation):
                if allocation[i, j + dj] >= 0:
                    next_moves.append((i, j + dj))
                    break
            else:
                break
        for dj in range(1, n_destinations):
            if is_within_bounds(i, j - dj, allocation):
                if allocation[i, j - dj] >= 0:
                    next_moves.append((i, j - dj))
                    break
            else:
                break
        if not next_moves:
            return find_next_moves(i, j, allocation, 'vertical')
    elif move_type == 'vertical':
        for di in range(1, n_sources):
            if is_within_bounds(i + di, j, allocation):
                if allocation[i + di, j] >= 0:
                    next_moves.append((i + di, j))
                    break
            else:
                break
        for di in range(1, n_sources):
            if is_within_bounds(i - di, j, allocation):
                if allocation[i - di, j] >= 0:
                    next_moves.append((i - di, j))
                    break
            else:
                break
        if not next_moves:
            return find_next_moves(i, j, allocation, 'horizontal')
    
    return next_moves

In [31]:
def find_next_move(i, j, allocation, visited, move_type, start):
    next_moves = find_next_moves(i, j, allocation, move_type)
    for move in next_moves:
        if move not in visited:
            return move
        elif move == start:
            return move
    
    # If no valid move is found, try the opposite direction
    opposite_move_type = 'vertical' if move_type == 'horizontal' else 'horizontal'
    next_moves = find_next_moves(i, j, allocation, opposite_move_type)
    for move in next_moves:
        if move not in visited:
            return move
        elif move == start:
            return move
    
    return None

In [32]:
def find_cycle(allocation_matrix, start_cell):
    logging.info(f"Starting to find a cycle from cell {start_cell}")
    n_sources, n_destinations = allocation_matrix.shape
    visited = set()
    visited.add(start_cell)
    current_cell = start_cell
    cycle = deque()
    cycle.append(start_cell)
    direction = 'horizontal'
    while True:
        next_move = find_next_move(current_cell[0], current_cell[1], allocation_matrix, visited, direction, start_cell)
        if next_move == start_cell:
            cycle.append(start_cell)
            break
        if next_move:
            cycle.append(next_move)
            visited.add(next_move)
            current_cell = next_move
            direction = 'vertical'
        else:
        # Find the next move in vertical direction
            next_move = find_next_move(current_cell[0], current_cell[1], allocation_matrix, visited, direction, start_cell)
            if next_move:
                cycle.append(next_move)
                visited.add(next_move)
                current_cell = next_move
                direction = 'horizontal'
            else:
                break
    return cycle

In [33]:
def remove_middle_points_from_cycle(cycle, start_cell):
    if not cycle:
        return cycle
    
    start_cell = cycle[0]
    row_points = {}
    col_points = {}
    
    # Collect points in rows and columns
    for (x, y) in cycle:
        if x not in row_points:
            row_points[x] = []
        row_points[x].append((x, y))
        
        if y not in col_points:
            col_points[y] = []
        col_points[y].append((x, y))
    
    # Remove middle points in rows, excluding the start cell's row
    for row, points in row_points.items():
        if len(points) > 2 and row != start_cell[0]:
            middle_points = points[1:-1]
            for point in middle_points:
                cycle.remove(point)
    
    # Remove middle points in columns, excluding the start cell's column
    for col, points in col_points.items():
        if len(points) > 2 and col != start_cell[1]:
            middle_points = points[1:-1]
            for point in middle_points:
                if point in cycle:
                    cycle.remove(point)
    
    return cycle

In [34]:
def adjust_allocation(allocation, i, j):
    logging.info(f"Adjusting allocation from Source {i+1}, Dest {j+1}...")
    n_rows, n_cols = allocation.shape
    visited = np.zeros_like(allocation, dtype=bool)
    loop = deque()    
    allocation[i, j] = 0
    
    n_nan = (n_rows * n_cols)-np.sum(np.isnan(allocation))
    while n_nan < n_rows + n_cols - 1:
        min_cost = float('inf')
        min_cost_idx = None
        for d in range(n_rows):
            for b in range(n_cols):
                if not(allocation[d, b] >= 0):
                    if cost[d][b] < min_cost:
                        min_cost = cost[d][b]
                        min_cost_idx = (d, b)
        allocation[min_cost_idx[0], min_cost_idx[1]] = 0
        n_nan += 1
    logging.info(f"Allocation matrix:\n{allocation}")
    cycle = find_cycle(allocation, (i,j))
    cycle = remove_middle_points_from_cycle(list(cycle), [i,j])
    logging.info(f"Cycle found: {cycle}")
    if cycle:
        pos, neg = [], []
        for index, (x, y) in enumerate(cycle):
            if index % 2 == 1:
                neg.append((x, y))
            else:
                pos.append((x, y))
        pos.pop()
        
        min_allocation = float('inf')
        loop_cells = []
        
        for (x, y) in neg:
            if allocation[x, y] is not None and allocation[x, y] >= 0:
                loop_cells.append((x, y))
                min_allocation = min(min_allocation, allocation[x, y])
        
        logging.info(f"Minus cells: {loop_cells}")
        logging.info(f"Minimum allocation in the minus cells: {min_allocation}")
        
        for (x, y) in pos:
            if not (allocation[x, y] >= 0):
                allocation[x, y] = 0
                
            allocation[x, y] += min_allocation
        for (x, y) in neg:
            if not (allocation[x, y] >= 0):
                allocation[x, y] = 0
            allocation[x, y] -= min_allocation
        
        logging.info(f"Updated allocation matrix:\n{allocation}")
    
    # get count of "nan" elements
    n_nan = temp = (n_rows * n_cols)-np.sum(np.isnan(allocation))
    while n_nan > n_rows + n_cols - 1:
        max_cost = -1
        max_cost_idx = None
        for d in range(n_rows):
            for b in range(n_cols):
                if allocation[d, b] == 0 and ([d, b] != [i, j]):
                    if cost[d][b] > max_cost:
                        max_cost = cost[d][b]
                        max_cost_idx = (d, b)
        allocation[max_cost_idx[0], max_cost_idx[1]] = None
        n_nan -= 1
        logging.info(f"Because Basis count = {temp} >= {n_rows + n_cols - 1} need to set cell {max_cost_idx} to 'nan' in the allocation matrix.")

    logging.info(f"Final allocation matrix:\n{allocation}")
    return allocation

In [35]:
def modi_method(allocation, cost):
    """
    Apply the MODI method to optimize the initial allocation.
    
    Parameters:
    - allocation: 2D array, the initial allocation matrix.
    - cost: 2D array, the cost matrix.
    
    Returns:
    - allocation: 2D array, the optimized allocation matrix.
    """
    # Convert cost to a NumPy array if it's not already
    cost = np.array(cost)
    n_rows, n_cols = allocation.shape

    for d in range(n_rows):
           for b in range(n_cols):
                if allocation[d, b] == 0:
                    allocation[d, b] = None
    while True:
        # Step 1: Calculate potentials (u, v)
        u, v = calculate_potentials(allocation, cost)
        
        # Step 2: Calculate opportunity costs (Delta_ij)
        delta = calculate_opportunity_costs(allocation, cost, u, v)
        
        # Step 3: Check for negative opportunity costs
        cell = find_most_negative_delta(delta)
        
        if cell is None:
            break  # No negative deltas, optimal solution found
        
        # Step 4: Adjust the allocation
        allocation = adjust_allocation(allocation, cell[0], cell[1])
    return allocation


In [36]:
for index, example in enumerate(data):
    logging.info(f"\nProcessing Example {index + 1}:")
    supply = example['supply']
    demand = example['demand']
    cost = example['cost']
    logging.info(f"Supply:\n\t{supply}")
    logging.info(f"Demand:\n\t{demand}")
    logging.info("Cost:")
    for a in cost:
        logging.info(f"\t{a}")
    # Get the allocation table using North-West Corner Method
    allocation = north_west_corner_method(supply, demand)

    # Calculate the total cost
    total_cost = calculate_total_cost(allocation, cost)
    logging.info(f"\n ~~~~~~~~~~~~~~~ Total Transportation Cost: {total_cost} ~~~~~~~~~~~~~~~")
    
    # Optimize the allocation using MODI method
    optimized_allocation = modi_method(allocation, cost)

    logging.info("Optimized Allocation Matrix:")
    logging.info(optimized_allocation)

    # Calculate the total transportation cost after optimization
    total_cost = calculate_total_cost(optimized_allocation, cost)
    logging.info(f"~~~~~~~~~~~~~~ Result: Optimized Total Transportation Cost: {total_cost}")